In [1]:
# for pep-8
!pip install nb_black
!pip install pycodestyle_magic
!pip install pycodestyle
!pip install flake8

  Using cached nb_black-1.0.7.tar.gz (4.8 kB)
  Created wheel for nb-black: filename=nb_black-1.0.7-py3-none-any.whl size=5297 sha256=f53a68ceb0bd74c0b45b5e2afd22bb8142c9c46683a00027c08d62b8e391b0a9
  Stored in directory: /home/experts-vision/.cache/pip/wheels/25/9d/fc/6312e3d8a66c164cc7f9d80bc66cc25de03a362a30b9f84aa3
Successfully built nb-black
  Using cached pycodestyle_magic-0.5-py2.py3-none-any.whl (9.5 kB)


In [2]:
%load_ext nb_black
%load_ext pycodestyle_magic

<IPython.core.display.Javascript object>

In [3]:
!pip install gdown

<IPython.core.display.Javascript object>

In [4]:
!pip install pandas numpy seaborn sklearn

  Using cached sklearn-0.0.post1.tar.gz (3.6 kB)
  Created wheel for sklearn: filename=sklearn-0.0.post1-py3-none-any.whl size=2959 sha256=84ad682743eb86532e199477780ca3a2de9aa918a353cbc22fafa7cf9acf7ef3
  Stored in directory: /home/experts-vision/.cache/pip/wheels/f8/e0/3d/9d0c2020c44a519b9f02ab4fa6d2a4a996c98d79ab2f569fa1
Successfully built sklearn


<IPython.core.display.Javascript object>

In [5]:
!pip install tensorflow

     |████████████████████████████████| 588.3 MB 64 kB/s  eta 0:00:011█████▌        | 431.6 MB 5.2 MB/s eta 0:00:31
  Using cached flatbuffers-22.10.26-py2.py3-none-any.whl (26 kB)
  Using cached opt_einsum-3.3.0-py3-none-any.whl (65 kB)
     |████████████████████████████████| 2.4 MB 3.0 MB/s eta 0:00:01
  Using cached absl_py-1.3.0-py3-none-any.whl (124 kB)
  Using cached gast-0.4.0-py3-none-any.whl (9.8 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
  Using cached tensorboard-2.11.0-py3-none-any.whl (6.0 MB)
  Using cached google_pasta-0.2.0-py3-none-any.whl (57 kB)
     |████████████████████████████████| 1.7 MB 2.9 MB/s eta 0:00:01
  Using cached libclang-14.0.6-py2.py3-none-manylinux2010_x86_64.whl (14.1 MB)
     |████████████████████████████████| 439 kB 2.9 MB/s eta 0:00:01
  Using cached tensorboard_plugin_wit-1.8.1-py3-none-any.whl (781 kB)
  Using cached tensorboard_data_server-0.6.1-py3-none-manylinux2010_x86_64.whl (4.9 MB)
  Using cached google_auth_oauthli

<IPython.core.display.Javascript object>

In [6]:
%reload_ext watermark
%watermark -v -p numpy,pandas,torch,transformers

ModuleNotFoundError: No module named 'watermark'

<IPython.core.display.Javascript object>

In [ ]:
import numpy as np
import time
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import seaborn as sns
from pylab import rcParams
from tqdm import tqdm
import matplotlib.pyplot as plt
from matplotlib import rc
from pandas.plotting import register_matplotlib_converters
from sklearn.model_selection import train_test_split
import tensorflow_hub as hub
import tensorflow_text
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from sklearn.preprocessing import LabelEncoder
import tensorflow_hub as hub
import json
import os
import zipfile

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

register_matplotlib_converters()
sns.set(style="whitegrid", palette="muted", font_scale=1.2)

HAPPY_COLORS_PALETTE = [
    "#01BEFE",
    "#FFDD00",
    "#FF7D00",
    "#FF006D",
    "#ADFF02",
    "#8F00FF",
]

sns.set_palette(sns.color_palette(HAPPY_COLORS_PALETTE))

rcParams["figure.figsize"] = 12, 8

RANDOM_SEED = 42

np.random.seed(RANDOM_SEED)
tf.random.set_seed(RANDOM_SEED)

# Download Universal Text Encoder


In [ ]:
use = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

In [ ]:
def embed(input):
    return use(input)


def plot_similarity(labels, features, rotation):
    corr = np.inner(features, features)
    sns.set(font_scale=1.2)
    g = sns.heatmap(
        corr, xticklabels=labels, yticklabels=labels, vmin=0, vmax=1, cmap="YlOrRd"
    )
    g.set_xticklabels(labels, rotation=rotation)
    g.set_title("Semantic Textual Similarity")


def run_and_plot(messages_):
    message_embeddings_ = embed(messages_)
    plot_similarity(messages_, message_embeddings_, 90)


messages = [
    # Smartphones
    "I like my phone",
    "My phone is not good.",
    "Your cellphone looks great.",
    # Weather
    "Will it snow tomorrow?",
    "Recently a lot of hurricanes have hit the US",
    "Global warming is real",
    # Food and health
    "An apple a day, keeps the doctors away",
    "Eating strawberries is healthy",
    "Is paleo better than keto?",
    # Asking about age
    "How old are you?",
    "what is your age?",
]

run_and_plot(messages)

# Data Exploration
I'm using the public dataset Financial PhraseBank (https://arxiv.org/abs/1307.5336).
It consists of 4845 english sentences selected randomly from financial news, labeled according to how the information contained might affect the mentioned company stock price: positive, negative or neutral.


## Download the dataset

In [ ]:
df = pd.read_csv("data_light/financial_news_dataset/financial_news_data.csv")

In [ ]:
df.columns = ["sentiment", "statement", "0", "1"]

In [ ]:
df = df.rename(
    columns={
        "neutral": "sentiment",
        "According to Gran , the company has no plans to move all production to Russia , although that is where the company is growing .": "statement",
    }
)

In [ ]:
df.columns

In [ ]:
df = df[["sentiment", "statement"]]

In [ ]:
df.head()

In [ ]:
df.info()

We have about 4845 examples. Let's check for missing values:

Great, no missing values in the score and review texts!

In [ ]:
sns.countplot(x="sentiment", data=df)
plt.xlabel("sentiment")

In [ ]:
from sklearn.preprocessing import OneHotEncoder

enc = OneHotEncoder(sparse=False)

In [ ]:
one_hot_enc = enc.fit_transform(df.sentiment.to_numpy().reshape(-1, 1))

In [ ]:
one_hot_enc[:10]

In [ ]:
enc.categories_

In [ ]:
enc.inverse_transform(one_hot_enc[:10])

In [ ]:
positive = [0, 0, 1]
negative = [1, 0, 0]
neutral = [0, 1, 0]

In [ ]:
train_reviews, test_reviews, y_train, y_test = train_test_split(
    df.statement, one_hot_enc, test_size=0.2, random_state=RANDOM_SEED
)

In [ ]:
X_train = []
for r in tqdm(train_reviews):
    emb = use([r])
    review_emb = tf.reshape(emb, [-1]).numpy()
    X_train.append(review_emb)

X_train = np.array(X_train)

In [ ]:
X_test = []
for r in tqdm(test_reviews):
    emb = use([r])
    review_emb = tf.reshape(emb, [-1]).numpy()
    X_test.append(review_emb)

X_test = np.array(X_test)

In [ ]:
X_train.shape, X_test.shape

In [ ]:
model = keras.Sequential()

model.add(
    keras.layers.Dense(units=256, input_shape=(X_train.shape[1],), activation="relu")
)
model.add(keras.layers.Dropout(rate=0.3))
model.add(keras.layers.Dense(units=128, activation="relu"))
model.add(keras.layers.Dropout(rate=0.5))
model.add(keras.layers.Dense(3, activation="softmax"))
model.compile(
    loss="categorical_crossentropy",
    optimizer=keras.optimizers.Adam(0.0001),
    metrics=["accuracy"],
)
model.summary()

In [ ]:
history = model.fit(
    X_train,
    y_train,
    epochs=15,
    batch_size=32,
    validation_split=0.1,
    verbose=1,
    shuffle=True,
)

In [ ]:
plt.plot(history.history["loss"], label="train loss")
plt.plot(history.history["val_loss"], label="val loss")
plt.xlabel("epoch")
plt.ylabel("Cross-entropy loss")
plt.legend()

In [ ]:
plt.plot(history.history["accuracy"], label="train accuracy")
plt.plot(history.history["val_accuracy"], label="val accuracy")
plt.xlabel("epoch")
plt.ylabel("accuracy")
plt.legend()

# Don't want to wait? Uncomment the next cell and download my pre-trained model:

In [ ]:
# from tensorflow.keras.models import load_model
# path = "data"
# file_name = "data/saved_models_use"
# if os.path.exists(file_name):
#     pass
# else:
#     isExist = os.path.exists(path)
#     if not isExist:

#         # Create a new directory because it does not exist
#         os.makedirs(path)
#         print("The new directory is created!")
#     !gdown  -O 'data/saved_models_use.zip' 1r2HWXpwJmoQcIqnEgJ7jKeQkuICpGaGE
#     with zipfile.ZipFile("data/saved_models_use.zip", "r") as zip_ref:
#         zip_ref.extractall("data")

In [ ]:
model.evaluate(X_test, y_test)[1]

In [ ]:
path = "data/saved_models_use"
if os.path.exists(path):
    pass
else:
    isExist = os.path.exists(path)
    if not isExist:

        # Create a new directory because it does not exist
        os.makedirs(path)
        print("The new directory is created!")

In [ ]:
model.save("data/saved_models_use")
print("Saved model to disk")

# Prediction on Raw Text

In [ ]:
# load model
model = load_model("data/saved_models_use")
# summarize model.
model.summary()

In [ ]:
test = "However , the rapidly increasing costs and the strengthening of the euro were challenging for the whole industry , and they impacted on our results."
emb = use([test])

result = model.predict(emb)

In [ ]:
result

In [ ]:
positive = [0, 0, 1]
negative = [1, 0, 0]
neutral = [0, 1, 0]

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

In [ ]:
class_names = ["negative", "neutral", "positive"]

In [ ]:
def show_confusion_matrix(confusion_matrix):
    hmap = sns.heatmap(confusion_matrix, annot=True, fmt="d", cmap="Blues")
    hmap.yaxis.set_ticklabels(hmap.yaxis.get_ticklabels(), rotation=0, ha="right")
    hmap.xaxis.set_ticklabels(hmap.xaxis.get_ticklabels(), rotation=30, ha="right")
    plt.ylabel("True sentiment")
    plt.xlabel("Predicted sentiment")


cm = confusion_matrix(y_test.argmax(axis=1), y_pred.argmax(axis=1))
df_cm = pd.DataFrame(cm, index=class_names, columns=class_names)
show_confusion_matrix(df_cm)

In [ ]:
list_test = df["statement"].tolist()

In [ ]:
%%time
result = []
for r in list_test:
    emb = use([r])
    pred=model.predict(emb)
    result.append(pred)